# MPC trajectory tracking


Consider the linearized model of the longitudinal movement of a quadcopter

$$m\ddot{p} = mg\theta, \\I_{\theta} \ddot{\theta} = \tau$$

where $p$ denotes the $x$ coordinated with respect to a world fixed frame, 
$\theta$ is the pitch angle, $\tau$ is

the torque, and $m$, $g$, and $I_{\theta}$ denote the mass, gravitation constant 
and the moment of inertia.

By scaling the pitch angle and the torque we can write the model as a fourth 
order integrator

$$\dot{x} = A_cx + Bu$$

where $x = \left[p \quad \dot{p} \quad \ddot{p} \quad \overset{\dots}p \right]$, 
$u = \overset{\dots.}p$, and

$$A_{c}=\left[\begin{array}{cccc}{0} & {1} & {0} & {0} \\ {0} & {0} & {1} 
& {0} \\ {0} & {0} & {0} & {1} \\ {0} & {0} & {0} & {0} \end{array}\right], 
\quad B_c = \left[\begin{array}{c} {0} \\ {0} \\{0} \\ {1} \end{array}\right].$$

In fact, note that $\ddot{p} = g\theta$ and $u = \frac{g}{I_{\theta}}\tau$. 
After zero order hold with sampling period $\tau$ and considering $x_k := x(k\tau)$ 
and

$u(t)=u_{k}, \quad t \in[k \tau,(k+1) \tau), \quad k \in\{0, \ldots, h-1\}$.

we can write

$$x_{k+1} = Ax_k + Bu_k$$

where $A = e^{A_c\tau}$, $B = \int_{0}^{\tau} e^{A_{c} s} ds B_c$.

Suppose that we wish to follow a give reference $r(t)$, which takes the values 
$r_k := r(k\tau)$ at sampling times and that the input is constraint to $|u_k| 
\leq c$.

To this effect, we use MPC with cost function at each time $k$,

$$\sum_{l=k}^{k+H-1} (p_k - r_k)^2 + \rho u_k^2$$

where $p_k = p(k\tau)$ and $H$ is the prediction horizon. The MPC algorithm 
runs for every $k \in \mathbb{N}_0$ (consider $h = \infty$). Consider $r(t) 
= 3 \sin(0.3t)$, $\rho = 0.01$, $\tau = 0.1$, $H = 80$, and $c = 0.4$. This 
amount to solving the following quadratic optimization problem at every time 
step $k$

$$\min (Mx_k+N\bar{u}-\bar{r})^T (Mx_k+N\bar{u}-\bar{r})+\rho\bar{u}^T\bar{u}$$

subject to 

$$\left[\matrix{ I \cr -I}\right]\bar{u}\leq c \left[\matrix{ 1 \cr 1 \cr 
\dots\cr 1}\right]$$

where $\bar{u} = [u_k \ \ u_{k+1} \ \ \dots u_{k+h-1}]^T$ $\bar{r} = [r_k 
\ \ r_{k+1} \ \ \dots r_{k+h-1}]^T$

$N=\left[\matrix{C A \cr CA^2 \cr \dots \cr CA^{H-k}}\right]$ $M=\left[\matrix{ 
CB          & 0           & 0           & 0     & \dots & 0     \cr
CAB         & CB          & 0           & 0     & \dots & 0     \cr 
CA^2B       & CAB         & CB          & 0     & \dots & 0     \cr 
CA^3B       & CA^2B       & CAB         & CB    & \dots & 0     \cr 
\dots       & \dots       & \dots       & \dots & \dots & \dots \cr
CA^{H-1-k}B & CA^{H-2-k}B & CA^{H-3-k}B & \dots & \dots & B
}\right]$

Note that the cost function can also be written as, apart from additive constant, 
as

$$\bar{u}^T (\rho I +N^T N ) \bar{u} + u^T (N^T(Mx_k-\bar{r})).$$

The next script implements this control policy for this system.

In [ ]:
import numpy as np
from scipy import signal
import scipy.linalg
import scipy.optimize
import matplotlib.pyplot as plt

In [ ]:
def solve_qp_scipy(H,f,A,b):
    def f_to_min(x):
        return 0.5 * np.dot(x, H).dot(x) + np.dot(f, x)
    
    lbnd = [-np.inf for idx in range(b.shape[0])]
    ubnd = [b[idx,0] for idx in range(b.shape[0])]
    constraints=scipy.optimize.LinearConstraint(A,lb=lbnd,ub=ubnd)

    result = scipy.optimize.minimize(f_to_min, x0=np.zeros(H.shape[0]) ,constraints=constraints)

    return result

In [ ]:
def mpcmatrices(A,B,C,H):
    n = A.shape[0]
    m = B.shape[1]
    p = C.shape[0]
    M = np.zeros((p*H,n))
    N = np.zeros((p*H,m*H))
    for i in range(H):
        M[1+(i-1)*p:p*i+1,:] = C@np.linalg.matrix_power(A, i+1)
        for j in range(i+1):
            N[1+(i-1)*p:p*i+1,1+(j-1)*m:m*j+1] = C@np.linalg.matrix_power(A,(i-j))@B
        
    return M, N

In [ ]:
#initial condition
opt = 3

if opt==1:
    X0 = np.array([[10], [0], [0], [0]])
elif opt==2:
    X0 = np.array([[-20], [-10], [0], [0]])
elif opt==3:
    X0 = np.array([[-1], [-1], [0.1], [0.4]])


#parameters 
tau = 0.1 #     sampling period
T   = 40 #      simulation time, seconds
H   = 80 #      MPC horizon steps
c = 0.4 # saturations

# definitions
# define 4th order integrator
Ac = np.array([[0, 1, 0, 0],[0, 0, 1, 0], [0, 0, 0, 1], [0, 0, 0, 0]])
Bc = np.array([[0], [0], [0], [1]])
Cc = np.array([[1, 0, 0, 0]]) # for mpc weight only position

#time
t = np.arange(0,T+tau,tau)
h = t.shape[0]-1

#reference
r  = 3
wr = 0.3
xr = r*np.sin(wr*t)
R = xr

#discretize
A, B, C = signal.cont2discrete((Ac, Bc, Cc, np.zeros(1)), tau)[:3]
n = A.shape[0]
m = B.shape[1]
p = C.shape[0]
rho = 0.01

#matrices for MPC
M, N = mpcmatrices(A,B,C,H)

#simulate system 
X = np.zeros((n,h))
u = np.zeros((m,h))
X[:,[0]] = X0
for k in range(h-1):
    # MPC
    if k <= (h-H-2):
        Rbar = np.atleast_2d(R[(k+1)*m : (k+1+H)*m]).T
        G = rho*np.eye(m*H)+N.T@N
        a = (N.T@(M@X[:,[k]]-Rbar)).T
        C_ = np.vstack((np.eye(m*H),-np.eye(m*H)))
        b = np.vstack((c*np.ones((m*H,1)),c*np.ones((m*H,1))))
        u_ = solve_qp_scipy(G, a, C_, b)    
        u[:,k] = u_.x[0:m]
    else:
        M, N = mpcmatrices(A, B, C, h-(k+1))
        Rbar = np.atleast_2d(R[ (k+1)*m : (h)*m]).T
        G = rho*np.eye(m*(h-(k+1)))+N.T@N
        a = (N.T@(M@X[:,[k]]-Rbar)).T
        C_ = np.vstack((np.eye(m*(h-(k+1))),-np.eye(m*(h-(k+1)))))
        b = np.vstack((c*np.ones((m*(h-(k+1)),1)),c*np.ones((m*(h-(k+1)),1))))
        u_ = solve_qp_scipy(G, a, C_, b)
        u[:,k] = u_.x[0:m]
    
    X[:,[k+1]] = A@X[:,[k]] + B*u[:,k]

In [ ]:
f = plt.figure()
ax = f.gca()
ax.plot(t[:h],X[0,:])
ax.plot(t,xr)
ax.set_xlabel(r'time $t = k\tau$')
ax.set_ylabel('p')
ax.legend(['p','r'])
ax.grid(True);

In [ ]:
f = plt.figure()
ax = f.gca()
ax.plot(t[:h],X[1,:])
ax.set_xlabel('time')
ax.set_ylabel('v')
ax.grid(True);

In [ ]:
f = plt.figure()
ax = f.gca()
ax.plot(t[:h],u[0,:])
ax.set_xlabel('time')
ax.set_ylabel('u')
ax.grid(True);